# File Download

In [85]:
import pandas as pd
import numpy as np

import datetime

In [86]:
today = pd.to_datetime(datetime.date.today())

year = today.year

ytd = pd.to_datetime(f'01-01-{year}')

t_minus_30 = today - datetime.timedelta(30)

t_minus_90 = today - datetime.timedelta(90)

In [87]:
ytd.value

1609459200000000000

In [112]:
tax_lot_df = pd.read_csv('/Users/DanCassin/Development/python/minervini_stock_selector/TTM_tax_lots_083021.csv',
            usecols=[
                'Symbol', 'Quantity', 'Opening Date', 'Cost/Share $', 'Total Cost $',
                'Closing Date',	'Price/Share $', 'Proceeds $', 'Gain $'],
            skiprows=13,
            delimiter=',',
            skipfooter=2,
            engine='python'
)

In [113]:
tax_lot_df

,Symbol,Quantity,Opening Date,Cost/Share $,Total Cost $,Closing Date,Price/Share $,Proceeds $,Gain $
0,AAPL,35.00000,--,--,4684.72,--,--,5119.857505,435.13
1,Sell,20.00000,06/17/2021,130.09,2601.75,07/16/2021,146.28,2925.632860,323.88
2,Sell,15.00000,07/03/2021,138.86,2082.98,07/16/2021,146.28,2194.224645,111.25
3,ACB,500.00000,--,--,4899.92,--,--,4563.898280,-336.02
4,Sell,80.00000,10/20/2020,4.36,349.20,03/11/2021,10.35,828.068000,478.87
...,...,...,...,...,...,...,...,...,...
275,Sell,0.13158,09/25/2020,42.03,5.53,03/30/2021,51.51,6.777808,1.25
276,Sell,0.08087,12/25/2020,48.84,3.95,03/30/2021,51.51,4.165689,0.22
277,YOU,70.00000,--,--,3480.85,--,--,3272.471720,-208.38
278,Sell,20.00000,08/23/2021,49.42,988.35,08/30/2021,46.75,934.991920,-53.36


In [114]:
tax_lot_df.rename(columns = lambda x: x.lower().replace('$', '').strip().replace(' ','_'), inplace=True)
#tax_lot_df.drop(['deferred_loss', 'term','lot_selection', 'unnamed:_12'], axis=1, inplace=True)

In [115]:
tax_lot_df = tax_lot_df.replace('--', np.nan)
tax_lot_df['symbol']= tax_lot_df['symbol'].str.strip()
tax_lot_df['symbol'].replace('Sell', np.nan, inplace=True)
tax_lot_df['symbol'] = tax_lot_df['symbol'].ffill()

In [116]:
tax_lot_df.dropna(subset=['opening_date'], inplace=True)

In [117]:
tax_lot_df['opening_date'] = tax_lot_df['opening_date'].astype('Datetime64')
tax_lot_df['closing_date'] = tax_lot_df['closing_date'].astype('Datetime64')

In [118]:
tax_lot_df['pct_gain/loss'] = round((tax_lot_df['gain'] / tax_lot_df['total_cost']) * 100, 2)

In [119]:
tax_lot_df.head()

,symbol,quantity,opening_date,cost/share,total_cost,closing_date,price/share,proceeds,gain,pct_gain/loss
1,AAPL,20.0,2021-06-17,130.09,2601.75,2021-07-16,146.28,2925.632860,323.88,12.45
2,AAPL,15.0,2021-07-03,138.86,2082.98,2021-07-16,146.28,2194.224645,111.25,5.34
4,ACB,80.0,2020-10-20,4.36,349.20,2021-03-11,10.35,828.068000,478.87,137.13
5,ACB,95.0,2021-03-22,9.85,935.49,2021-04-30,8.89,845.009230,-90.48,-9.67
6,ACB,100.0,2021-03-04,9.83,982.78,2021-04-30,8.89,889.483400,-93.30,-9.49


In [ ]:
tax_lot_df['pct_gain/loss'].hist(bins=50);

In [ ]:
tax_lot_df['pct_gain/loss'].mean()

In [ ]:
tax_lot_df['pct_gain/loss'].median()

In [ ]:
tax_lot_df['days_held'] = (tax_lot_df['closing_date'] - tax_lot_df['opening_date']).dt.days

In [ ]:
tax_lot_df['days_held'].hist(bins=50)

In [ ]:
sub_df = tax_lot_df[['pct_gain/loss','days_held']]

sub_df = sub_df[sub_df['pct_gain/loss'] <=100]

sub_df.plot(kind='scatter', x='days_held', y='pct_gain/loss')

In [ ]:
print(tax_lot_df['days_held'].mean())
print(tax_lot_df['days_held'].median())

In [ ]:
tax_lot_df['days_held']

In [ ]:
last_90_days = tax_lot_df[tax_lot_df['opening_date'] > pd.to_datetime(t_minus_90)]

In [ ]:
last_90_days['pct_gain/loss'].hist()

In [ ]:
last_30_days = tax_lot_df[tax_lot_df['opening_date'] > pd.to_datetime(t_minus_30)]

last_30_days['pct_gain/loss'].hist()

In [ ]:
last_30_losses = last_30_days[last_30_days['pct_gain/loss'] < 0]

round(last_30_losses['gain'].sum(),2)

In [ ]:
last_30_gains = last_30_days[last_30_days['pct_gain/loss'] > 0]

last_30_gains['gain'].sum()

In [ ]:
total_trades_90_days = int(last_90_days.shape[0])

pct_gain_trades_90 = (last_90_days[last_90_days['gain'] > 0].shape[0] / total_trades_90_days) * 100

total_trades_30_days = int(last_30_days.shape[0])

pct_gain_trades_30 = (last_30_days[last_30_days['gain'] > 0].shape[0] / total_trades_30_days) * 100

print(f'Profitable Trades Last 30 Days:\n{round(pct_gain_trades_30, 2)}%')

print(f'Profitable Trades Last 90 Days:\n{round(pct_gain_trades_90, 2)}%')

In [ ]:
winning_trades = tax_lot_df[tax_lot_df['gain']> 0]
losing_trades = tax_lot_df[tax_lot_df['gain'] < 0]

total_winning_trades = winning_trades.shape[0]
total_losing_trades = losing_trades.shape[0]

avg_gain = round(winning_trades['gain'].mean(), 2)
avg_loss = abs(round(losing_trades['gain'].mean(), 2))

risk_reward = round((total_winning_trades * avg_gain) / (total_losing_trades * avg_loss), 2)

print(f'Reward/Risk {risk_reward}')

# Additions

- Comparison of last 30 days to last 90, YTD, TTM
